In [4]:
from yucca.image_processing.transforms.Spatial import Spatial
from yucca.image_processing.transforms.BiasField import BiasField
from yucca.image_processing.transforms.Noise import AdditiveNoise, MultiplicativeNoise
from yucca.image_processing.transforms.Ghosting import MotionGhosting
from yucca.image_processing.transforms.Ringing import GibbsRinging
from nibabel.testing import data_path
import nibabel as nib
import os
from copy import deepcopy
import numpy as np
import timeit

np.random.seed(4215235)
iterations = 2
# We use a publicly available sample from Nibabel
example_file = os.path.join(data_path, 'example4d.nii.gz')
im = nib.load(example_file)
imarr = im.get_fdata()[:, :, :, 0]


# If you want to use a local example for dataset specific times, 
# uncomment the following and supply the path to a local image the imarr variable

#local_file = 'YOUR/PATH/HERE'
#im = nib.load(local_file)
#imarr = im.get_fdata()

# Convert it to the format expected of the transforms
# a dict of {"image": image, "seg": segmentation}
# with samples for dims (b, c, x, y, z) for 3D or (b, c, x, y) for 2D
# In this case we use slice 12 for an image of (128, 96, 24) - adapt this as you please.
print("3D shape: ", imarr.shape)
imarr = imarr[np.newaxis, np.newaxis]
imarr = np.vstack([imarr]*2)
seg = np.zeros(imarr.shape)
datadict = {"image": imarr, "label": seg}
print("3D shape with batch and channel dimension: ", imarr.shape)



3D shape:  (128, 96, 24)
3D shape with batch and channel dimension:  (2, 1, 128, 96, 24)


In [5]:
idx = np.array(imarr.shape[2:])//4
imarr_half = imarr[:, :, idx[0]:-idx[0], idx[1]:-idx[1], idx[2]:-idx[2]]
seg_half = np.zeros(imarr_half.shape)
datadict_half = {"image": imarr_half, "label": seg_half}

In [6]:
# Applying Rotation
tform = Spatial(x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45))
print("Rotation: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)

Rotation:  0.4538204999998925 
  Rotation half:  0.04848909999986972


In [7]:
# Applying Deformation
from yucca.image_processing.transforms.Spatial import Spatial

tform = Spatial(deform_alpha=(700, 700),
                deform_sigma=(10, 10))
print("Deformation: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Deformation half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Deformation:  0.2743619000000308 
  Deformation half:  0.05162790000008499


In [8]:
# Applying Scaling
tform = Spatial(scale_factor=(1.5, 1.5))
print("Scaling: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Scaling half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)

Scaling:  0.3436143499998252 
  Scaling half:  0.045838400000093316


In [9]:
# Applying Rotation, Deformation and Scaling WITHOUT cropping
tform = Spatial(x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                scale_factor=(1.5, 1.5))
print("Rotation, Deformation and Scaling WITHOUT cropping: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation, Deformation and Scaling WITHOUT cropping half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Rotation, Deformation and Scaling WITHOUT cropping:  0.2800752000000557 
  Rotation, Deformation and Scaling WITHOUT cropping half:  0.0313183000000663


In [10]:
# Applying Rotation, Deformation and Scaling WITH cropping
tform = Spatial(crop=True,
                patch_size=np.array(imarr.shape[2:])//2,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                scale_factor=(1.5, 1.5))
# Applying Rotation, Deformation and Scaling WITH cropping
tform2 = Spatial(crop=True,
                patch_size=np.array(imarr_half.shape[2:])//2,
                x_rot_in_degrees=(0, 0),
                y_rot_in_degrees=(0, 0),
                z_rot_in_degrees=(45, 45),
                deform_alpha=(700, 700),
                deform_sigma=(10, 10),
                scale_factor=(1.5, 1.5))
print("Rotation, Deformation and Scaling WITH cropping: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Rotation, Deformation and Scaling WITH cropping half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Rotation, Deformation and Scaling WITH cropping:  0.06260975000009239 
  Rotation, Deformation and Scaling WITH cropping half:  0.030556599999954415


In [11]:
# Applying Additive Noise
tform = AdditiveNoise(p_per_sample=1, mean=(0, 0),
                      sigma=(25, 25))
print("Additive Noise: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Additive Noise half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Additive Noise:  0.030095100000153252 
  Additive Noise half:  0.004527999999936583


In [12]:

# Applying Multiplicative Noise
tform = MultiplicativeNoise(p_per_sample=1, mean=(0, 0),
                            sigma=(0.1, 0.1))
print("Multiplicative Noise: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Multiplicative Noise half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)



Multiplicative Noise:  0.054680250000046726 
  Multiplicative Noise half:  0.005203549999805546


In [13]:
# Applying Bias Field
tform = BiasField(p_per_sample=1)
print("Bias Field: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Bias Field half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Bias Field:  0.11863205000008747 
  Bias Field half:  0.013049900000169146


In [15]:
# Applying Motion Ghosting
tform = MotionGhosting(p_per_sample=1, alpha=(0.6, 0.6),
                       numReps=(3, 4),
                       axes=(0, 1))
print("Motion Ghosting: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Motion Ghosting half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Motion Ghosting:  0.13636829999995825 
  Motion Ghosting half:  0.015186950000042998


In [17]:

# Applying Gibbs Ringing
tform = GibbsRinging(p_per_sample=1, cutFreq=(30, 36),
                     axes=(0, 1))
print("Gibbs Ringing: ", timeit.timeit(lambda: tform(**deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Gibbs Ringing half: ", timeit.timeit(lambda: tform(**deepcopy(datadict_half)), number=iterations)/iterations)


Gibbs Ringing:  0.13577650000001995 
  Gibbs Ringing half:  0.01198219999992034
